Welcome to part 2 of the alanine tutorial. Here we take the computed dataset we used previously and solve for site specific delta values. We begin by importing.

In [1]:
import sys; sys.path.insert(0, '..')

In [2]:
import alanineTest
import readInput as ri
import fragmentAndSimulate as fas
import solveSystem as ss
import basicDeltaOperations as op

import copy
from tqdm import tqdm
import solveSystem as ss
import numpy as np
import sympy as sy

Here, we have imported the alanineTest.py file which we discussed before, allowing us to predict what the dataset should look like for a given standard. We standardize by defining and computing a forward model, without experimental fractionation and with all peaks observed, giving us theoretical values for each peak. To do so, we need to specify some hypothesized standard structure, which will often be wrong; it turns out that sample standard comparisons are robust to reasonable natural abundance errors in our hypothesized standard, so this does not need to be perfect. We encourage the reader to read the theory paper and look at tests 4 and 5 for more details. 

In [3]:
deltas = [-30,-30,0,0,0,0]
fragSubset = ['full','44']
df, expandedFrags, fragSubgeometryKeys, fragmentationDictionary = alanineTest.initializeAlanine(deltas, fragSubset)

forbiddenPeaks = {}

Delta 18O
0.0


In [4]:
predictedMeasurement, MNDictStd, FF = alanineTest.simulateMeasurement(df, fragmentationDictionary, expandedFrags, 
                                                                      fragSubgeometryKeys, 
                                                                       abundanceThreshold = 0,
                                                                       MNMassThreshold = 1,
                                                                         unresolvedDict = {},
                                                                        outputFull = False)

100%|███████████████████████████████████████████████████████████████████████████| 1512/1512 [00:00<00:00, 17017.44it/s]

Calculating Isotopologue Concentrations
Compiling Isotopologue Dictionary
Simulating Measurement


Next, we read in our sample and standard files. We have specific functions in the readInput.py for doing so; note that different functions are used for importing experimental data. The error option allows us to specify an error for each observed peak of sample and standard, e.g. a 1 per mil error on each observed peak. Likely experimental results will have different errors for different beams, but this is a basic way to understand how error will propagate. We can set error again for the molecular average measurement, from ri.SampleUValues

In [6]:
standardJSON = ri.readJSON("2021-06-28 TUTORIAL Standard Stochastic.json")
processStandard = ri.readComputedData(standardJSON, error = 0, theory = predictedMeasurement)

sampleJSON = ri.readJSON("2021-06-28 TUTORIAL Sample Stochastic.json")
processSample = ri.readComputedData(sampleJSON, error = 0)
UValuesSmp = ri.readComputedUValues(sampleJSON, error = 0)

In [7]:
processStandard

{'M1': {'full': {'Subs': ['D', '15N', '17O', '13C'],
   'Predicted Abundance': [0.03246510641983099,
    0.09577373138104994,
    0.019795669505800258,
    0.8519654926933189],
   'Observed Abundance': [0.03246510641983099,
    0.09577373138104994,
    0.019795669505800258,
    0.8519654926933189],
   'Error': [0.0, 0.0, 0.0, 0.0]},
  '44': {'Subs': ['Unsub', 'D', '15N', '13C'],
   'Predicted Abundance': [0.3119004436751976,
    0.024348829814873244,
    0.09577373138104994,
    0.5679769951288792],
   'Observed Abundance': [0.3119004436751976,
    0.024348829814873244,
    0.09577373138104994,
    0.5679769951288792],
   'Error': [0.0, 0.0, 0.0, 0.0]}}}

Now, we set up the matrix inversion problem to find site-specific information. At this point, it would be useful to review the theory paper's description of this process. Computationally, we first determine which isotopologues were introduced via this experiment and track where they appear in the different fragments. We include a "precise identity" specifying in words which isotopologue these correspond to. 

In [8]:
MNKey = "M1"
isotopologuesDict = fas.isotopologueDataFrame(MNDictStd, df)
Isotopologues = isotopologuesDict[MNKey]
Isotopologues

,Number,Full,Stochastic,Mass,Composition,Stochastic U,full_01 Identity,full_01 Subs,44_01 Identity,44_01 Subs,Precise Identity
00100000000000,1,"(0, 0)1(0, 0)0(0, 0, 0, 0, 0, 0)(0, 0)",0.01045,1,13C,0.0109,00100000000000,13C,00xxx0000000xx,Unsub,13C Ccarboxyl
01000000000000,2,"(0, 1)0(0, 0)0(0, 0, 0, 0, 0, 0)(0, 0)",0.020899,1,13C,0.0218,01000000000000,13C,01xxx0000000xx,13C,13C Calphabeta
00000100000000,1,"(0, 0)0(0, 0)1(0, 0, 0, 0, 0, 0)(0, 0)",0.003524,1,15N,0.003676,00000100000000,15N,00xxx1000000xx,15N,15N Namine
00001000000000,2,"(0, 0)0(0, 1)0(0, 0, 0, 0, 0, 0)(0, 0)",0.000728,1,17O,0.00076,00001000000000,17O,00xxx0000000xx,Unsub,17O Ocarboxyl
00000000000001,2,"(0, 0)0(0, 0)0(0, 0, 0, 0, 0, 0)(0, 1)",0.000299,1,D,0.000312,00000000000001,D,00xxx0000000xx,Unsub,D Hlost
00000000000100,6,"(0, 0)0(0, 0)0(0, 0, 0, 0, 0, 1)(0, 0)",0.000896,1,D,0.000935,00000000000100,D,00xxx0000001xx,D,D Hretained


Next, we define "Percent Abundance correction factors". We don't discuss these in detail here--users should read the theory paper and look at tests 6-8. The measurement we simulated so far for this tutorial was perfect, so here all correction factors are 1. 

In [9]:
pACorrection = ss.percentAbundanceCorrectTheoretical(predictedMeasurement, processSample, massThreshold = 1)

In [10]:
pACorrection

{'M1': {'full': 1.0, '44': 1.0}}

To solve and propagate error, we perform a monte carlo routine. We will go through the steps of this routine now. 

First, we perturb our PA factors (again, see theory paper). We then perturb our standard based on their observed errors. Following perturbation, we calculate correction factors by taking the ratio between the perturbed and predicted abundance. 

In [11]:
variablePACorrect = copy.deepcopy(pACorrection)

In [12]:
variablePACorrect = ss.modifyPercentAbundanceCorrection(pACorrection, variablePACorrect, MNKey, amount = 0)
std = ss.perturbStandard(processStandard, theory = True)

In [13]:
std

{'M1': {'full': {'Subs': ['D', '15N', '17O', '13C'],
   'Predicted Abundance': [0.03246510641983099,
    0.09577373138104994,
    0.019795669505800258,
    0.8519654926933189],
   'Observed Abundance': [0.03246510641983099,
    0.09577373138104994,
    0.019795669505800258,
    0.8519654926933189],
   'Error': [0.0, 0.0, 0.0, 0.0],
   'Perturbed': array([0.03246511, 0.09577373, 0.01979567, 0.85196549]),
   'Correction Factor': array([1., 1., 1., 1.])},
  '44': {'Subs': ['Unsub', 'D', '15N', '13C'],
   'Predicted Abundance': [0.3119004436751976,
    0.024348829814873244,
    0.09577373138104994,
    0.5679769951288792],
   'Observed Abundance': [0.3119004436751976,
    0.024348829814873244,
    0.09577373138104994,
    0.5679769951288792],
   'Error': [0.0, 0.0, 0.0, 0.0],
   'Perturbed': array([0.31190044, 0.02434883, 0.09577373, 0.567977  ]),
   'Correction Factor': array([1., 1., 1., 1.])}}}

Next, we perturb our sample. This process has a few subroutines; we first discuss those, then show how they are run in a single function. Our first subroutine perturbs based on experimental error, as is done with the standard. 

In [14]:
perturbSample = ss.perturbSampleError(processSample)
perturbSample

{'M1': {'full': {'Observed Abundance': array([0.03349737, 0.09509673, 0.01985428, 0.85155163]),
   'Subs': ['D', '15N', '17O', '13C']},
  '44': {'Observed Abundance': array([0.31608592, 0.02503145, 0.09509673, 0.5637859 ]),
   'Subs': ['Unsub', 'D', '15N', '13C']}}}

Next, we apply correction factors calculated from the standard to the sample. renormalize should be set to be True; advanced users who are familiar with TEST 9 may wish to set it to False. 

In [15]:
correctedSample = ss.perturbSampleCorrectionFactors(perturbSample, std, renormalize = True)
correctedSample

{'M1': {'full': {'Observed Abundance': array([0.03349737, 0.09509673, 0.01985428, 0.85155163]),
   'Subs': ['D', '15N', '17O', '13C']},
  '44': {'Observed Abundance': array([0.31608592, 0.02503145, 0.09509673, 0.5637859 ]),
   'Subs': ['Unsub', 'D', '15N', '13C']}}}

Finally, we apply our U Value correction factors, scaling our observations based on the hypothesized abundances of unobserved peaks. The output of this function gives the data we will use for the matrix routine.

In [16]:
pACorrectedSample = ss.perturbSamplePACorrection(correctedSample, variablePACorrect)
pACorrectedSample

{'M1': {'full': {'Observed Abundance': array([0.03349737, 0.09509673, 0.01985428, 0.85155163]),
   'Subs': ['D', '15N', '17O', '13C']},
  '44': {'Observed Abundance': array([0.31608592, 0.02503145, 0.09509673, 0.5637859 ]),
   'Subs': ['Unsub', 'D', '15N', '13C']}}}

Rather than run these functions individually each time, we have a parent function which deals with all of the sample perturbation. This function additionally processes the UValueCorrectedSample variables to give dataframes, and allows us to procedurally turn on or off different correction factors. Advanced users can go into more detail with these by reading the relevant sections of the paper and looking at the function description. 

The M1 entry includes a corrected, standardized relative abundance for each observed ion beam. 

In [17]:
smp = ss.perturbSample(processSample, std, variablePACorrect, experimentalPACorrectList = [],
                   pACorrect = True, WCorrect = True)['M1']
smp

,full,44
D,0.033497,0.025031
15N,0.095097,0.095097
17O,0.019854,0.000000
13C,0.851552,0.563786
Unsub,0.000000,0.316086


Next, we use knowledge of which isotopologues fragment to yield which substitutions and the corrected, standardized relative abundances to set up the matrix problem we hope to invert. The "comp" variable gives the composition matrix, specifying how each isotopologue is sampled in each observation. The columns of this matrix correspond to the Isotopologues in the Isotopologues dataframe; the first column is 13C Ccarboxyl, the second is 13C Calphabeta, and so forth. The rows refer to different observations; the first gives closure, then the second gives the full.D observation, the third gives full.15N, the fourth full.17O, and the fifth full.13C; the pattern repeats for the 44 peak. The rows of the "meas" vector are given in the same way. 

In [18]:
comp, meas = ss.constructMatrix(Isotopologues, smp, MNKey, fragmentationDictionary,
                                    subMatrix = [])

In [19]:
sy.Matrix(comp)

Matrix([
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
[0.0, 0.0, 0.0, 0.0, 1.0, 1.0],
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
[1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
[1.0, 0.0, 0.0, 1.0, 1.0, 0.0]])

In [20]:
sy.Matrix(meas)

Matrix([
[               1.0],
[ 0.033497365197376],
[ 0.095096727401987],
[0.0198542809707603],
[ 0.851551626429877],
[0.0250314454391084],
[ 0.095096727401987],
[ 0.563785904394953],
[ 0.316085922763952]])

We can solve this matrix inversion multiple ways. First, we can use the np.linalg.lstsq routine, which is most useful for fully constrained systems. Alternatively, we can run a Gauss-Jordan elimination algorithm; this is useful for underconstrained systems, as it can help us determine which individual isotopologues are unsolved for, and which are well constrained.

In [21]:
sol = np.linalg.lstsq(comp, meas, rcond = -1)
sol[0]

array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
       0.02503145])

In [22]:
AugMatrix = np.column_stack((comp, meas))
solve = ss.GJElim(AugMatrix, augMatrix = True)
sy.Matrix(solve[0])

Matrix([
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0,    0.287765722034924],
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0,    0.563785904394953],
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0,    0.095096727401987],
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0,   0.0198542809707603],
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0,  0.00846591975826753],
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0,   0.0250314454391084],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0,                  0.0],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.46944695195361e-17],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  1.2490009027033e-16]])

This entire process is performed by the M1MonteCarlo function, which perturbs U Values, standard, and sample, then solves the system, for N steps. The options GJ and debugMatrix can both be set to True in order to see output from every step of the GJ Solution, which may help advanced users troubleshoot. They should look at the code to see exactly what this function is outputting in that case. 

In [23]:
M1Results = ss.M1MonteCarlo(processStandard, processSample, pACorrection, isotopologuesDict,
                            fragmentationDictionary, experimentalPACorrectList = [], 
                            N = 100, GJ = False, debugMatrix = False, pACorrect = True, WCorrect = True,
                           perturbTheoryPAAmt = 0)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 109.63it/s]


In [24]:
M1Results

{'GJ': [],
 'NUMPY': [array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
         0.02503145]),
  array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
         0.02503145]),
  array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
         0.02503145]),
  array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
         0.02503145]),
  array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
         0.02503145]),
  array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
         0.02503145]),
  array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
         0.02503145]),
  array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
         0.02503145]),
  array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
         0.02503145]),
  array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.00846592,
         0.02503145]),
  array([0.28776572, 0.5637859 , 0.09509673, 0.01985428, 0.0

Next, we need to process these results and make sense of them. The key mathematical step here is going from percent abundance space (where our solution is now) into U Value space, making use of the U^M+N variable. We encourage the user to review the relevant parts of the theory paper here. We will perform all of these steps for each solution to the Monte Carlo process; as an example, we take one solution and demonstrate the process. 

We first take all the isotopologues that were introduced and assign their percent abundances from the solution. 

In [25]:
out = isotopologuesDict['M1'][['Number','Stochastic','Composition','Stochastic U','Precise Identity']].copy()
out['M1 Percent Abundance'] = M1Results['NUMPY'][0]
out

,Number,Stochastic,Composition,Stochastic U,Precise Identity,M1 Percent Abundance
00100000000000,1,0.01045,13C,0.0109,13C Ccarboxyl,0.287766
01000000000000,2,0.020899,13C,0.0218,13C Calphabeta,0.563786
00000100000000,1,0.003524,15N,0.003676,15N Namine,0.095097
00001000000000,2,0.000728,17O,0.00076,17O Ocarboxyl,0.019854
00000000000001,2,0.000299,D,0.000312,D Hlost,0.008466
00000000000100,6,0.000896,D,0.000935,D Hretained,0.025031


Then, we calculate the U^M+1 value. The process for this is elaborated in the M+N theory paper; briefly, we take the observed U Value for some isotope of interest and divide by the sum of the percent abundances of all isotopologues with that isotopic composition. For example, we would take the 13C U value and divide by the M1 Percent Abundance of 13C Ccarboxyl + 13C Calphabeta.

We may do this for multiple substitutions; the ones we will do it for are determined by the UMNSub parameter, a list of U Values to use. If we calculate it multiple ways, we take the average and use this as our U^M+1.

In [26]:
#Perturb U Values
UPerturb = ss.PerturbUValue(UValuesSmp)

#Calculate UM1
UM1 = ss.calcUMN(MNKey, out, UPerturb, UMNSub = ['13C'])
out['UM1'] = UM1
out['Calc U Values'] = out['M1 Percent Abundance'] * out['UM1']
out

,Number,Stochastic,Composition,Stochastic U,Precise Identity,M1 Percent Abundance,UM1,Calc U Values
00100000000000,1,0.01045,13C,0.0109,13C Ccarboxyl,0.287766,0.038269,0.011012
01000000000000,2,0.020899,13C,0.0218,13C Calphabeta,0.563786,0.038269,0.021575
00000100000000,1,0.003524,15N,0.003676,15N Namine,0.095097,0.038269,0.003639
00001000000000,2,0.000728,17O,0.00076,17O Ocarboxyl,0.019854,0.038269,0.000760
00000000000001,2,0.000299,D,0.000312,D Hlost,0.008466,0.038269,0.000324
00000000000100,6,0.000896,D,0.000935,D Hretained,0.025031,0.038269,0.000958


We can compute delta values directly using the Isotopologues dataframe. But it would be convenient to calculate them for our initial input dataframe, which lists the sites in a different order. So we next process this information to follow the same order as our input dataframe. Then we normalize fo the number of atoms, and compute both the absolute delta in PDB etc. space and the relative sample standard delta. The "absolute delta" will generally be incorrect unless we have perfect knowledge of the standard; the relative delta will generally be precise (see TEST 3, 10). 

Keep in mind that relative deltas are not directly additive--if the sample is -40, and the standard is -30, the relative delta will not be precisely -10!

In [27]:
#This section reassigns the solutions of the isotopologues dataframe to the right order for the 
#site-specific dataframe
M1 = [0] * len(out.index)
UM1 = [0] * len(out.index)
U = [0] * len(out.index)
for i, v in out.iterrows():
    identity = v['Precise Identity'].split(' ')[1]
    index = list(df.index).index(identity)

    M1[index] = v['M1 Percent Abundance']
    UM1[index] = v['UM1']
    U[index] = v['Calc U Values']
    

#calculate relevant information
normM1 = U / df['Number']
#This gives deltas in absolute reference frame
smpDeltasAbs = [op.ratioToDelta(x,y) for x, y in zip(df['IDS'], normM1)]

appxStd = df['deltas']

#This gives deltas relative to standard
relSmpStdDeltas = [op.compareRelDelta(atomID, delta1, delta2) for atomID, delta1, delta2 in zip(df['IDS'], appxStd, smpDeltasAbs)]

relSmpStdDeltas

[-10.309278350514983,
 10.309278350514983,
 -4.440892098500626e-12,
 -9.999999999998789,
 24.999999999990585,
 39.99999999998649]

As before, we define a single function that takes care of all of this, repeating the process for the N results from the Monte Carlo model. 

In [28]:
processedResults = ss.processM1MCResults(M1Results, UValuesSmp, isotopologuesDict, df, 
                                         UMNSub = ['13C'])

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 207.46it/s]


In [29]:
processedResults

{'PDB etc. Deltas': [[-39.99999999999948,
   -20.000000000000682,
   -4.551914400963142e-12,
   -9.999999999998677,
   24.999999999990365,
   39.99999999998671],
  [-39.99999999999948,
   -20.000000000000682,
   -4.551914400963142e-12,
   -9.999999999998677,
   24.999999999990365,
   39.99999999998671],
  [-39.99999999999948,
   -20.000000000000682,
   -4.551914400963142e-12,
   -9.999999999998677,
   24.999999999990365,
   39.99999999998671],
  [-39.99999999999948,
   -20.000000000000682,
   -4.551914400963142e-12,
   -9.999999999998677,
   24.999999999990365,
   39.99999999998671],
  [-39.99999999999948,
   -20.000000000000682,
   -4.551914400963142e-12,
   -9.999999999998677,
   24.999999999990365,
   39.99999999998671],
  [-39.99999999999948,
   -20.000000000000682,
   -4.551914400963142e-12,
   -9.999999999998677,
   24.999999999990365,
   39.99999999998671],
  [-39.99999999999948,
   -20.000000000000682,
   -4.551914400963142e-12,
   -9.999999999998677,
   24.999999999990365,
   

Finally, we update the original dataframe with these answers, calculating means and errors (standard deviations) for each value. 

In [30]:
ss.updateSiteSpecificDfM1MC(processedResults, df)

,IDS,Number,deltas,full_01,44_01,PDB etc. Deltas,PDB etc. Deltas Error,Relative Deltas,Relative Deltas Error,M1 Percent Abundance,M1 Percent Abundance Error,UM1,UM1 Error,Calc U Values,Calc U Values Error
Calphabeta,C,2,-30,1,1,-4.000000e+01,4.263256e-14,-1.030928e+01,1.065814e-14,0.563786,7.771561e-16,0.038269,6.245005e-17,0.021575,5.898060e-17
Ccarboxyl,C,1,-30,1,x,-2.000000e+01,0.000000e+00,1.030928e+01,1.065814e-14,0.287766,2.775558e-16,0.038269,6.245005e-17,0.011012,1.040834e-17
Ocarboxyl,O,2,0,1,x,-4.551914e-12,0.000000e+00,-4.440892e-12,0.000000e+00,0.019854,2.081668e-17,0.038269,6.245005e-17,0.000760,1.084202e-19
Namine,N,1,0,1,1,-1.000000e+01,1.776357e-14,-1.000000e+01,1.598721e-14,0.095097,1.249001e-16,0.038269,6.245005e-17,0.003639,2.602085e-18
Hretained,H,6,0,1,1,2.500000e+01,3.907985e-14,2.500000e+01,7.105427e-15,0.025031,1.387779e-17,0.038269,6.245005e-17,0.000958,1.626303e-18
Hlost,H,2,0,1,x,4.000000e+01,7.105427e-14,4.000000e+01,4.263256e-14,0.008466,1.734723e-17,0.038269,6.245005e-17,0.000324,3.252607e-19
